In [ ]:
%pip install pygame
!pip install pyserial
!pip install numpy
!pip install websockets
!pip install keyboard
!pip install nest_asyncio

import pygame
import math
import serial
import numpy
import asyncio
import nest_asyncio
import websockets
import keyboard

# Apply asyncio fix for Jupyter
nest_asyncio.apply()

pygame.init()

# Constants based on lidar resolution
LIDAR_RESOLUTION = 80
VISUALIZATION_RESOLUTION = 80

# Global variables
distances_list = []
mode = 'm'
bs = 0
first_run = True

async def recv(websocket):
    global distances_list, first_run
    async for message in websocket:
        d = message.split(",")[:-1]
        if first_run:
            distances_list = d
            first_run = False
        else:
            distances_list = filterDistancesList(d)
        await GameUpdate()
        await websocket.send(checkKeys())

def GenerateLinePositions(numberOfLines):
    angle = 360 / numberOfLines
    lines = []
    for x in range(numberOfLines):
        lines.append([
            300 * math.cos(x * angle / 180 * math.pi),
            300 * math.sin(x * angle / 180 * math.pi)
        ])
    return lines

def checkKeys():
    global mode
    if keyboard.is_pressed('a'):
        mode = 'a'
    elif keyboard.is_pressed('m'):
        mode = 'm'

    if mode == 'a':
        return directionAlgorithm()
    elif mode == 'm':
        if keyboard.is_pressed('up'):
            return 'forward'
        elif keyboard.is_pressed('down'):
            return 'stop'
        elif keyboard.is_pressed('left'):
            return 'left'
        elif keyboard.is_pressed('right'):
            return 'right'
        else:
            return 'stop'

def directionAlgorithm():
    global bs
    alg_dir = 'stop'
    biggestSpace = [0, 0]
    for a in range(int(len(distances_list)/2)-10):
        sum = 1
        for b in range(10):
            sum += int(distances_list[2*a + 2*b]) * int(distances_list[2*a + 2*b + 1])
        if biggestSpace[1] < sum:
            biggestSpace[0] = a + 5
            biggestSpace[1] = sum
            bs = a * 2
    if biggestSpace[0] > 60 or biggestSpace[0] < 20:
        alg_dir = 'left'
    elif 20 < biggestSpace[0] < 50:
        alg_dir = 'right'
    else:
        alg_dir = 'forward'
    return alg_dir

def filterDistancesList(d_list):
    filtered_list = []
    for i in range(int(len(d_list)/2)):
        if int(d_list[i*2 + 1]) < 100:
            filtered_list.append(distances_list[i*2])
            filtered_list.append(distances_list[i*2 + 1])
        elif int(d_list[i*2]) > 7500:
            filtered_list.append(distances_list[i*2])
            filtered_list.append(distances_list[i*2 + 1])
        else:
            filtered_list.append(d_list[i*2])
            filtered_list.append(d_list[i*2 + 1])
    return filtered_list

def map(val, in_min, in_max, out_min, out_max):
    return (val - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

# Generate line positions
line_positions = GenerateLinePositions(VISUALIZATION_RESOLUTION)

# Set up the drawing window
screen = pygame.display.set_mode([800, 800])
sysfont = pygame.font.get_default_font()
font1 = pygame.font.SysFont(sysfont, 72)

async def GameUpdate():
    if len(distances_list) == VISUALIZATION_RESOLUTION * 2:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

        screen.fill((20, 10, 30))

        for x in range(VISUALIZATION_RESOLUTION):
            a = int(distances_list[x*2]) / 2000
            if (mode == 'a') and int(map(bs, 0, 160, 80, 0)) == x:
                pygame.draw.aaline(screen, (0, 255, 0), (400, 400), (line_positions[x][0]+400, line_positions[x][1]+400), 4)
            else:
                brightness = int(distances_list[x*2 + 1])
                pygame.draw.aaline(screen, (brightness, brightness, brightness), (400, 400), (line_positions[x][0]*a+400, line_positions[x][1]*a+400), 4)

        pygame.draw.circle(screen, pygame.Color(100, 100, 120), (400, 400), 60)
        pygame.display.flip()
        pygame.image.save(screen, "map_image.png")

async def main():
    async with websockets.serve(recv, "192.168.0.107", 8084):
        await asyncio.Future()  # Run forever

# Start server in notebook environment
loop = asyncio.get_event_loop()
loop.create_task(main())


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Task pending name='Task-1' coro=<main() running at C:\Users\User\AppData\Local\Temp\ipykernel_6604\504718009.py:140>>

: 